In [60]:
import speech_recognition as sr
import os
from playsound import playsound
import time
import openai
import weaviate
import json
import requests
import pyttsx3

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'

api_key = os.environ.get('OPENAI_APIKEY')
openai.api_key = api_key

GPT_MODEL = "gpt-3.5-turbo-0613"

weaviate_client_url = "http://localhost:8080"

client = weaviate.Client(
    url=weaviate_client_url, 
    additional_headers={
        "X-OpenAI-Api-Key": api_key
    }
)

tester_engine = pyttsx3.init()
tester_engine.setProperty('rate', 175)
tester_engine.setProperty('voice', tester_engine.getProperty('voices')[1].id)

# Create a recognizer object
r = sr.Recognizer()

# Prompt the user to speak
print("Please speak now...")
tester_engine.say("Please speak now")
tester_engine.runAndWait()
tester_engine.stop()

with sr.Microphone() as source:
    audio = r.listen(source)
    tester_engine.say("Input received")
    tester_engine.runAndWait()
    tester_engine.stop()
print("Processing...")

start_time = time.time()


Please speak now...
Processing...


In [61]:
try:
    # Convert speech to text
    text = r.recognize_google(audio)

except sr.UnknownValueError:
    tester_engine.say("Sorry, I was unable to understand")
    tester_engine.runAndWait()
    tester_engine.stop() 

print("Your text input is ", text)

Your text input is  how is tea different from coffee


In [62]:
phrase_prompt = """
What are the keywords or phrases from the sentence :
"""+text+"\nOutput:"
print("The input prompt is")
print(phrase_prompt)

response = openai.ChatCompletion.create(
  model=GPT_MODEL,
  messages=[
    {"role" : "system", "content" : "You are to find upto 3 keywords or keyphrases from a given sentence"},
    {"role" : "system", "content" : "Restrict yourself to ONLY provide the answer, and nothing else"},
    {"role" : "system", "content" : "Provide the response as a list of words which are comma separated"},
    {"role" : "user", "content" : phrase_prompt},
  ]
)

keyphrases = response["choices"][0]["message"]["content"]
print(keyphrases)

The input prompt is

What are the keywords or phrases from the sentence :
how is tea different from coffee
Output:
tea, different, coffee


In [63]:
# context is essentially a proof of concept rn

contents = list()
title = list()
context = "Context Below -> "
keywords = keyphrases.split(",")
keywords = [word.strip() for word in keywords]

print(keywords)

nearText = {"concepts" : text}
nearResponse = (
    client.query
    .get("Article", ["title", "content"])
    .with_near_text(nearText)
    .with_limit(1)
    .do()
)

wiki_content = nearResponse['data']['Get']['Article'][0]['content']
lines = wiki_content.split('.')

wiki_title = nearResponse['data']['Get']['Article'][0]['title']
context = context + "\n" + wiki_title + "\n"

for line in lines[0:2]:
    if line == "":
        break
    context = context + line + "\n"
    print(line)

for keyword in keywords:
    response = (
        client.query
        .get("Article", ["title", "content"])
        .with_hybrid(keyword, alpha=0.5)  # default 0.75
        .with_limit(1)
        .do()
    )
    
    if response == nearResponse:
        continue

    wiki_content = response['data']['Get']['Article'][0]['content']
    lines = wiki_content.split('.')
    
    wiki_title = response['data']['Get']['Article'][0]['title']
    context = context + "\n" + wiki_title + "\n"
    
    for line in lines[0:2]:
        if line[0] == "\n":
            break
        context = context + line + "\n"
context = context + "\n<- Context Above"
print(context)

['tea', 'different', 'coffee']
Tea is a drink that is made from the steeping the leaves of the Camellia sinensis plant
 Tea can have other herbs, spices, or fruit flavors in it, like lemon
Context Below -> 
Tea
Tea is a drink that is made from the steeping the leaves of the Camellia sinensis plant
 Tea can have other herbs, spices, or fruit flavors in it, like lemon

Japanese tea ceremony
The Japanese tea ceremony (called cha-no-yu, chado, or sado) is a special way of making green tea (matcha 抹茶)
  It is called the Way of Tea

Allotropy
Allotropy (or allotropism) is when a chemical element can exist in two or more different forms in the same physical state or phase
 These different forms are called allotropes

Coffee
Coffee is a plant (Coffea) and the name of the drink that is made from this plant
 The coffee plant is a bush or tree that can grow up to ten meters (about 32 feet) high, but is usually cut shorter

<- Context Above


In [70]:
personality = """
You are a personal assistant. Your job is to answer the user queries succintly and in under 100 words.
You will be provided some additional info from a database, and have to decide if given info is relevant to the query.
If you used the context, start your response with 'Sure!', else start your response with 'Alright'."""
user_prompt = text

print("\nThe input prompt is")
print(user_prompt)
print("\nGPT Instructions:")
print(personality)
print("\nContext is given as:")
print(context)


The input prompt is
how is tea different from coffee

GPT Instructions:

You are a personal assistant. Your job is to answer the user queries succintly and in under 100 words.
You will be provided some additional info from a database, and have to decide if given info is relevant to the query.
If you used the context, start your response with 'Sure!', else start your response with 'Alright'.

Context is given as:
Context Below -> 
Tea
Tea is a drink that is made from the steeping the leaves of the Camellia sinensis plant
 Tea can have other herbs, spices, or fruit flavors in it, like lemon

Japanese tea ceremony
The Japanese tea ceremony (called cha-no-yu, chado, or sado) is a special way of making green tea (matcha 抹茶)
  It is called the Way of Tea

Allotropy
Allotropy (or allotropism) is when a chemical element can exist in two or more different forms in the same physical state or phase
 These different forms are called allotropes

Coffee
Coffee is a plant (Coffea) and the name of th

In [71]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city, eg. San Francisco, or Chennai",
                },
            },
            "required": ["location"],
        },
    }]

def get_current_weather(location):
    # print("test")
    # print(location)
    # print(format)
    api_key = os.environ.get('weather_api_key')
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    complete_url = base_url + "appid=" + api_key + "&q=" + location
    response = requests.get(complete_url)
    x = response.json()
    if x["cod"] != "404":
        y = x["main"]
        current_temperature = y["temp"]
        z = x['weather']
        weather_description = z[0]["description"]
        return ("The current temperature is "+str(round(current_temperature-273.15, 2))+"Celsius and the weather is "+str(weather_description))

function_exec = {
    'get_current_weather' : get_current_weather
}

In [72]:
query_response = openai.ChatCompletion.create(
  model=GPT_MODEL,
  messages=[
    {"role" : "system", "content" : personality+context},
    {"role" : "user", "content" : user_prompt}
  ],
  functions=functions
)

In [73]:
choices = query_response['choices'][0]
finish = choices['finish_reason']
if finish == 'function_call':
  func_params = choices['message']['function_call']
  func_name = func_params['name']
  func_inputs = json.loads(func_params['arguments'])
  answer = function_exec[choices['message']['function_call']['name']](func_inputs.get("location"))
  
  for sentences in answer.split(". "):
    print(sentences)
    
else:
  answer = choices['message']['content']
  for sentences in answer.split(". "):
    print(sentences)

Tea and coffee are different in terms of their source and preparation
Tea is made from the leaves of the Camellia sinensis plant, while coffee is made from the beans of the Coffea plant
Tea is typically steeped in hot water, while coffee is brewed by passing hot water through ground coffee beans
Additionally, tea contains caffeine but in smaller amounts compared to coffee
The taste and aroma of tea and coffee also differ, with tea often having a milder flavor and coffee being bolder and more robust.


In [68]:
tester_engine.say(answer)
tester_engine.runAndWait()
tester_engine.stop()

end_time = time.time()

print("time taken : ", -start_time + end_time)

time taken :  42.71615552902222


In [69]:
# ramblings ->

# # alright crazy idea
# instead of some bigass RAKE up my codebase or doing logn NLTK stuff
# what if i just .... 
# use openAI for keyword extraction?????
# god i may be a genius (re: Sarcasm)

# shifting model to 3.5 turbo, saves SOO many tokens
# also chat completion instead of simple completion
# already trained, just need to piepline

# docker-compose up -d <- to create
# docker-compose start <- to start
# docker-compose down <- to remove and exit
# docker-compose stop <- to stop

# some_objects = client.data_object.get()
# # print(json.dumps(some_objects))
# print("Article title  : " + some_objects['objects'][0]['properties']['title'])
# print("Article sample : " + some_objects['objects'][0]['properties']['content'][0:80])

# do i need a 'personality' ??
# or is the default fine?
# im kinda not making a neuro-mini anymore
# just doing my own thing
# remind myself -> rethink personality inputs

# Run TTS on the text
# tts.tts_to_file(reply, file_path='output.wav')
# second idea, can i maybe stream RVC through this?
# or SOVITS

